In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('newsis_all.pkl')

In [3]:
len(df)

3019885

## 섹션 분리

In [4]:
df['sec']=df['section'].apply(lambda x: (x.split('|')[0]).strip())
df['sec_sub']=df['section'].apply(lambda x: (x.split('|')[1]).strip())

In [5]:
df=df[df['sec']!='']

In [6]:
df=df.reset_index(drop=True)

In [7]:
len(df)

3005495

In [24]:
df_in=df[df['sec']=='스포츠']

In [25]:
counts=df_in.groupby('sec_sub').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec_sub
스포츠일반     93754
야구/MLB    61934
축구        43994
농구        24825
골프        21425
야구        17594
배구         9389
해외야구        868
평창올림픽       781
해외축구        610
리우올림픽       576
dtype: int64

In [16]:
sorted_counts.to_csv('./newsis_life.csv')

In [17]:
df_in.to_pickle('newsis_local.pkl')

## 필요없는 데이터 제거

In [11]:
df.drop(['author', 'date','section'], axis = 1, inplace = True)

## 본문 길이 체크

In [12]:
df['article']=df['article'].apply(lambda x: x.strip())
df['len']=df['article'].apply(lambda x: len(x))
df=df[df['len']>0]

df=df.reset_index(drop=True)

In [13]:
len(df)

3004642

In [10]:
df.tail(4)

,article,title,sec,len
3004638,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,더 신중하게,스포츠,85
3004639,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,멀리 멀리,스포츠,85
3004640,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,어느 위치가 좋을까,스포츠,84
3004641,1일 경상북도 칠곡군 왜관에 위치한 파미힐스CC에서 열린 2017 DGB금융그룹 대...,포즈취하는 정한밀,스포츠,85


In [ ]:
df.to_pickle('newsis_sec.pkl')

## 섹션 정리

In [4]:
df = pd.read_pickle('newsis_sec.pkl')

In [5]:
counts=df.groupby('sec').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec
사회      911719
전국      573195
정치      540389
경제      305404
스포츠     275750
국제      220694
연예       77112
문화       56073
산업       33739
사람들       5277
광장        2784
기타        1113
라이프       1035
오피니언       345
국감          13
dtype: int64

In [6]:
df_soc=df[df['sec']=='사회']
docs = df_soc.article

In [7]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]


CPU times: user 20 ms, sys: 12 ms, total: 32 ms
Wall time: 79.3 ms


In [8]:
stopwords_newsis=['뒤', '곳', '군', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [9]:
%%time
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_newsis, ngram_range=(1,3), min_df=2, max_df=.5)
vect_weights = vect.fit_transform(docs)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

CPU times: user 23min 49s, sys: 10.9 s, total: 24min
Wall time: 24min 5s


In [10]:
weights_df.sort_values(by='weight', ascending=False).head(50)

,term,weight
3210021,시민,0.004835
394734,경찰,0.004553
4828978,종로구,0.004389
2788196,서울 종로구,0.004293
1466972,대통령,0.004075
1038462,기자,0.003920
4919443,중,0.003898
6214104,호,0.003776
6171911,혐의,0.003775
5936105,학교,0.003744


In [4]:
counts=df.groupby('sec').size()
section_idx=enumerate(counts.keys())
section_to_int = {word: ii for ii, word in section_idx}
section_to_int

{'경제': 0,
 '광장': 1,
 '국감': 2,
 '국제': 3,
 '기타': 4,
 '라이프': 5,
 '문화': 6,
 '사람들': 7,
 '사회': 8,
 '산업': 9,
 '스포츠': 10,
 '연예': 11,
 '오피니언': 12,
 '전국': 13,
 '정치': 14}

In [ ]:
def set_section_idx(section):
    return section_to_int[section]

In [ ]:
pickle_dict1['sec_idx']=pickle_dict1['sec'].apply(set_section_idx)


## 문장추출

In [3]:
doc_sentences=df['article'].str.findall(r"[0-9a-zA-Z가-힣%,()\- ]+\.")

In [4]:
len(doc_sentences)

3004642

In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
pos = lambda d: ['/'.join(p) for p in m.pos(d)]

tagged_sentences=doc_sentences.apply(lambda x:pos(''.join(x)))

KeyboardInterrupt: 

In [6]:
words_count=tagged_sentences.apply(lambda x:len(x))

In [7]:
df['sentences']=doc_sentences
df['tagged_sentences']=tagged_sentences
df['words_count']=words_count

In [10]:
df[df['words_count']==0]

,article,title,sec,len,sec_idx,sentences,tagged_sentences,words_count
1,"2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스...",6.25전쟁 사진전 보는 학생들,사회,123,8,[],[],0
2,1일 오전 서울 동대문구청사에서 열린 학교급식 식재료의 질적 개선과 안정적인 공급을...,'어느 쌀이 우수할까',사회,89,8,[],[],0
7,김석균(왼쪽) 해양경찰청장이 1일 오후 서해·제주 해역에서 광역초계기(CN-235)...,"김석균 해양경찰청장, 외국어선 조업실태 점검",사회,81,8,[],[],0
13,1일 오후 경기도 고양의 어울림누리 빙상장에서 열린 ‘2013-2014 아시아리그 ...,센스 있는 심판!,스포츠,97,10,[],[],0
15,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,취임사하는 서근우 신임 이사장,경제,73,0,[],[],0
17,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,사기 흔드는 서근우 이사장,경제,73,0,[],[],0
18,2일 오후 서울 마포구 공덕동 신용보증기금에서 열린 신용보증기금 서근우 신임 이사장...,취임사하는 서근우 신임 이사장,경제,73,0,[],[],0
25,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,역투하는 한화 이브랜드,스포츠,81,10,[],[],0
26,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,선취점 얻어내는 LG,스포츠,126,10,[],[],0
27,2일 오후 서울 잠실야구장에서 열린 2013 한국야쿠르트 세븐 프로야구 한화 이글스...,"LG 김기태 감독, '좋았어'",스포츠,127,10,[],[],0


In [28]:
df.iloc[1,0]

"2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스티벌'에서 (사)월드피스자유연합 (대표 안재철)이 6.25전쟁 사진전과 6.25전쟁 당시 대한민국을 지원한 67개국 국기를 전시하고 있다."

In [26]:
df[df.index==1].article

1    2일부터 오는 6일까지 계룡대에서 개최되는 '2013년 계룡 군문화축제, 지상군페스...
Name: article, dtype: object

In [47]:
df.reset_index(level=0, inplace=True)

In [49]:
df.drop(['index'], axis = 1, inplace = True)

In [51]:
df.to_pickle('kbo_parsed.pkl')

In [90]:
df['title']=df['title'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+", " ")
df['contents']=df['contents'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+|[0-9a-zA-Z.]+.kr|[0-9a-zA-Z.]+.com|[0-9a-zA-Z.]+@[a-zA-Z.]+|[0-9a-zA-Z.]+@|\([a-z.:/]+\)|마이데일리.|조이뉴스24.|[a-zA-Z,. ]+\.|기사제공 [0-9a-zA-Z가-힣 .]", " ")

In [96]:
df['contents']=df['contents'].str.findall(r"[0-9a-zA-Z가-힣,.!?<>() ]+\. ").apply(lambda x: ' '.join(x))